<a href="https://colab.research.google.com/github/pedrootrevisan/SensoriamentoRemoto/blob/main/NBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import pandas

In [9]:
ee.Authenticate()
ee.Initialize(project = 'ee-pedrotrevisanoliveira')


In [32]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
#area de interesse
roi = ee.Geometry.Point([-49.473477438123425, -18.952055288852442]);

In [34]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                                  .filterBounds(roi)\
                                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1))\
                                  .filterDate('2020-01-01','2021-01-01')
print('qtd informações',collection.size().getInfo())

qtd informações 21


In [35]:
melhor_imagem = collection.sort('CLOUDY_PIXEL_PERCENTAGE',True).first()
print('ID', melhor_imagem.get('PRODUCT_ID').getInfo())
print('% nuvens',melhor_imagem.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())

image_scale = melhor_imagem.select('B.*').multiply(0.0001)


ID S2B_MSIL2A_20200904T133229_N0214_R081_T22KFE_20200908T100034
% nuvens 0.006269


In [36]:
map.addLayer(image_scale,{'bands':['B4','B3','B2'],'min':0.019, 'max':0.19},'RGB Sentinel')
map.addLayer(image_scale,{'bands':['B8','B4','B3'],'min':0.019, 'max':0.19},'False Color')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [31]:
nbr = image_scale.normalizedDifference(['B8','B12']).rename('nbr')
map.addLayer(nbr,{'palette': ['red','orange','yellow','green','darkgreen'],'min':0,'max':0.8},'NBR')
map

Map(bottom=289685.0, center=[-18.381243705060864, -49.63760375976563], controls=(WidgetControl(options=['posit…